In [1]:
import tracemalloc
from queryConstruction import constructQuery
import globalVars
from ranker import *
from clustering import *
from searchIndexer import *
tracemalloc.start()
print("Start Running: ",tracemalloc.get_traced_memory())

print("loading data")
listOfQueries = getListQueries()
path = globalVars.path

##########load schema###########
with open(path+'/TestSchemas/sportsSchema.pickle','rb') as file:
    testSchema = pickle.load(file)
    
print("testSchema: ",tracemalloc.get_traced_memory())
tracemalloc.stop()

tracemalloc.start()
###########one hot encoding############
globalVars.init()

OneHotVocab = globalVars.OneHotVocab 

Start Running:  (576, 12562)
loading data
testSchema:  (150511247, 158070149)


In [2]:
#####get max number of entities in dataset and get total number of unique attributes ###
import json
datasets = ["cosql","greatSql","nvBench","sparc","splash","topics"]
maxEntities = 0
totalSelectAttrs = 0
totalWhereAttrs = 0
d = ""
l = {}
for dataset in datasets:
    with open('/home/nada/GP/GP/GP/notebooks/preparingDatasets/finalOutputs/'+dataset+'.json', 'r',encoding='UTF-8') as file:
        print(dataset)
        data = json.load(file)
        for query in data:
            for q in query["allQueries"]:
                totalSelectAttrs = totalSelectAttrs + len(list(set(q["selectAttrs"])))
                totalWhereAttrs = totalWhereAttrs + len(list(set(q["whereAttrs"])))
                if len(q["entities"]) > maxEntities:
                    maxEntities = len(q["entities"])
                    l = q


print(maxEntities)
print(totalSelectAttrs)
print(totalWhereAttrs)
print(l)

cosql
greatSql
nvBench
sparc
splash
topics
23
127414
78849
{'query': 'select distinct farealias0.fare_id from airport_service as airport_servicealias0 , airport_service as airport_servicealias1 , city as cityalias0 , city as cityalias1 , days as daysalias0 , days as daysalias1 , days as daysalias2 , days as daysalias3 , days as daysalias4 , days as daysalias5 , days as daysalias6 , days as daysalias7 , days as daysalias8 , days as daysalias9 , fare as farealias0 , fare_basis as fare_basisalias0 , fare_basis as fare_basisalias1 , fare_basis as fare_basisalias2 , fare_basis as fare_basisalias3 , fare_basis as fare_basisalias4 , fare_basis as fare_basisalias5 , flight as flightalias0 , flight_fare as flight_farealias0 where ( ( ( ( ( ( ( ( ( ( daysalias8.day_name = " day_name0 " and daysalias9.day_name = " day_name1 " and flightalias0.flight_days = daysalias8.days_code and flightalias0.flight_days = daysalias9.days_code ) and daysalias7.day_name = " day_name2 " and flightalias0.flight_day

In [4]:
# 1-gram => prob of selecting attribute in general
# 2-gram => prob of selecting attribute given entity
# 3-gram => prob of selecting attribute given 2 entities 
# and so on .....
# max number of entities 12 => 13-gram 


In [4]:
#####get unigram (calculating frequencies) for attributes ###
import json
datasets = ["cosql","greatSql","nvBench","sparc","splash","topics"]
ngramsDict = {}
ngramsDict["selectAttrsDict"] = {}
ngramsDict["whereAtrrsDict"] = {}
for dataset in datasets:
    with open('/home/nada/GP/GP/GP/notebooks/preparingDatasets/finalOutputs/'+dataset+'.json', 'r',encoding='UTF-8') as file:
        print(dataset)
        data = json.load(file)
        for query in data:
            for q in query["allQueries"]:
                for attr in q["selectAttrs"]:
                    if attr.find(".") != -1:
                        attr = attr.split(".")[1]
                    attr = attr.split("_")
                    attrOneHotVector = (getKeyWordsVector(attr)).tostring()
                    if ngramsDict["selectAttrsDict"].get(attrOneHotVector) is None:
                        ngramsDict["selectAttrsDict"][attrOneHotVector] = 1
                    else:
                        ngramsDict["selectAttrsDict"][attrOneHotVector] += 1
                for attr in q["whereAttrs"]:
                    if attr.find(".") != -1:
                        attr = attr.split(".")[1]
                    attr = attr.split("_")
                    attrOneHotVector = (getKeyWordsVector(attr)).tostring()
                    if ngramsDict["whereAtrrsDict"].get(attrOneHotVector) is None:
                        ngramsDict["whereAtrrsDict"][attrOneHotVector] = 1
                    else:
                        ngramsDict["whereAtrrsDict"][attrOneHotVector] += 1


cosql


/home/nada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/home/nada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


greatSql
nvBench
sparc
splash
topics


In [8]:
import pickle

with open("/home/nihal/Desktop/new_repo/GP/src/SearchEngine/nGrams/0.pickle", 'wb') as handle:
    pickle.dump(ngramsDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
def createNgramsDict(attributes,name,combinations):
    for combination in combinations:
        for attr in list(set(attributes)):
            if attr.find(".") != -1:
                attr = attr.split(".")[1]
            attr = attr.split("_")
            attrOneHotVector = (getKeyWordsVector(attr)).tostring()
            if nGramsDict[name][np.array(list(combination)).tostring()].get(attrOneHotVector) is None:
                nGramsDict[name][np.array(list(combination)).tostring()][attrOneHotVector] = 0
            nGramsDict[name][np.array(list(combination)).tostring()][attrOneHotVector] += 1

In [4]:
import json
import itertools
import numpy as np
import pickle
nGramsDict = {}
nGramsDict["selectAttrsDict"] = {}
nGramsDict["whereAtrrsDict"] = {}
datasets = ["cosql","greatSql","nvBench","sparc","splash","academic","advising","geography","imdb","restaurants","scholar","spider","yelp"]
entitiesProp={}
for dataset in datasets:

    with open('/home/nada/GP/GP/GP/notebooks/preparingDatasets/finalOutputs/'+dataset+'.json', 'r',encoding='UTF-8') as file:
        print(dataset)
        data = json.load(file)
        for query in data:
            for q in query["allQueries"]:
                entitiesOneHotVector = []
                combinations = []
                for entity in list(set(q["entities"])):
                    entity = entity.split("_")
                    entityOneHotVector = (getKeyWordsVector(entity)).tostring()
                    entityKey = np.array(list(entityOneHotVector)).tostring()
                    if entitiesProp.get(entityKey) is None:
                        entitiesProp[entityKey] = 0
                    entitiesProp[entityKey] += 1
                    entitiesOneHotVector.append(entityOneHotVector)
                for r in range(len(entitiesOneHotVector)+1):
                    for combination in itertools.combinations(entitiesOneHotVector, r):
                        if len(list(combination)) > 0:
                            combinations.append(list(combination))
                            nGramsDict["selectAttrsDict"][np.array(list(combination)).tostring()] = {}
                            nGramsDict["whereAtrrsDict"][np.array(list(combination)).tostring()] = {}
                whereAttributes = [attr[0] for attr in q["whereAttrs"]]
                createNgramsDict(q["selectAttrs"],"selectAttrsDict",combinations)
                createNgramsDict(whereAttributes,"whereAtrrsDict",combinations)
                        


cosql


/home/nada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/home/nada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/home/nada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/home/nada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
/home/nada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  import sys
/home/nada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  
/home/nada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


greatSql
nvBench
sparc
splash
academic
advising
geography
imdb
restaurants
scholar
spider
yelp


In [5]:
import pickle
with open("/home/nada/GP/GP/GP/src/SearchEngine/nGrams/ngrams.pickle", 'wb') as handle:
    pickle.dump(nGramsDict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
nGramsDict={}

In [ ]:
import pickle
with open("/home/nada/GP/GP/GP/src/SearchEngine/nGrams/entities.pickle", 'wb') as handle:
    pickle.dump(entitiesProp, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [4]:
import pickle
ngramFile = open("/home/nada/GP/GP/GP/src/SearchEngine/nGrams/ngrams.pickle",'rb')
nGramsDict = pickle.load(ngramFile)